**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/pick.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Let's do all of our imports here, too.
import numpy as np

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, ConnectMeshcatVisualizer, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MultibodyPlant, MultibodyPositionToGeometryPose, Parser, 
    PiecewisePolynomial, PiecewiseQuaternionSlerp, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource
)
from pydrake.examples.manipulation_station import ManipulationStation


In [ ]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step = 0.0)
parser = Parser(plant, scene_graph)
grasp = parser.AddModelFromFile(FindResourceOrThrow(
    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "grasp")
# TODO(russt): Draw the pregrasp gripper, too, as transparent (drake #13970).
#pregrasp = parser.AddModelFromFile(FindResourceOrThrow(
#    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "pregrasp")
brick = parser.AddModelFromFile(FindResourceOrThrow(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf"), "brick")
plant.Finalize()

frames_to_draw = {"grasp": {"body"},
                  "brick": {"base_link"}}
meshcat = ConnectMeshcatVisualizer(builder,
    scene_graph,
    zmq_url=zmq_url,
    frames_to_draw=frames_to_draw,
    axis_length=0.3,
    axis_radius=0.01)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

# TODO(russt): Set a random pose of the object.

# Get the current object, O, pose
B_O = plant.GetBodyByName("base_link", brick)
X_WO = plant.EvalBodyPoseInWorld(plant_context, B_O)

B_Ggrasp = plant.GetBodyByName("body", grasp)
p_GgraspO = [0, 0.12, 0]
R_GgraspO = RotationMatrix.MakeXRotation(np.pi/2.0).multiply(
    RotationMatrix.MakeZRotation(np.pi/2.0))
# Useful for a better image:
p_GgraspO = [0, 0.3, 0.1]
R_GgraspO = R_GgraspO.multiply(RotationMatrix.MakeYRotation(0.5))
X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
X_OGgrasp = X_GgraspO.inverse()
X_WGgrasp = X_WO.multiply(X_OGgrasp)

print(f"p_GO_W = {X_WGgrasp.rotation().multiply(X_GgraspO.translation())}")
print(f"p_GO_G = {X_GgraspO.translation()}")
print(f"p_OG_O = {X_OGgrasp.translation()}")

plant.SetFreeBodyPose(plant_context, B_Ggrasp, X_WGgrasp)
# Open the fingers, too.
plant.GetJointByName("left_finger_sliding_joint", grasp).set_translation(plant_context, -0.054)
plant.GetJointByName("right_finger_sliding_joint", grasp).set_translation(plant_context, 0.054)

meshcat.load()
diagram.Publish(context)